In [22]:
import csv
import pandas as pd
from collections import Counter
from sklearn.naive_bayes import MultinomialNB      
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.svm import LinearSVC

In [4]:
#Lendo o arquivo csv com o pandas
df = pd.read_csv('./dados/site_alura.csv', encoding="utf-8")

In [6]:
"""
Separando as colunas do csv entre:
X = Características do dados
Y = Classificação das características 
"""

X = df[["recencia", "frequencia", "semanas"]]
Y = df["situacao"]

In [ ]:
X_dummies = pd.get_dummies(X)
X = X_dummies.values

In [26]:
porcentagem_treino = 0.6
porcentagem_teste = 0.2

tamanho_treino = int(porcentagem_treino * len(X))
tamanho_teste = int(porcentagem_teste * len(X))
tamanho_validacao = len(Y) - (tamanho_treino + tamanho_teste)

treino_dados = X[0:tamanho_treino]
treino_marcacoes = Y[0: tamanho_treino]

fim_dados_teste = tamanho_treino + tamanho_teste
teste_dados = X[tamanho_treino:fim_dados_teste]
teste_marcacoes = Y[tamanho_treino:fim_dados_teste]

validacao_dados = X[fim_dados_teste:]
validacao_marcacoes = Y[fim_dados_teste:]

In [27]:
print("Tamanho de treino: ", tamanho_treino)
print("Tamanho de teste: ", tamanho_teste)
print("Tamanho de validação: ", tamanho_validacao)

Tamanho de treino:  135
Tamanho de teste:  45
Tamanho de validação:  45


In [28]:
"""Verificando a acuracia do modelo, comparando com modelos burros"""

contador = Counter(teste_marcacoes)
taxa_acerto_base = 100.0 * max(Counter(teste_marcacoes).values())/len(teste_marcacoes)

print("Contador: ", contador)
print("Taxa de acertos base: {:.2f}" .format(taxa_acerto_base))

Contador:  Counter({1: 36, 2: 6, 0: 3})
Taxa de acertos base: 80.00


In [17]:
#1º Separação 90% para treino 10% para teste, acuracia 82,00
#2º Separação 80% para treino 20% para teste, acuracia 82,00
#3º Separação 70% para treino 30% para teste, acuracia 83,66
#4º Separação 60% para treino 40% para teste, acuracia 84,25
#5º Separação 50% para treino 50% para teste, acuracia 82,19

def fit_and_predict(modelo, nome_algoritmo, treino_dados, treino_marcacoes, teste_dados, teste_marcadores):
    """
        Treinando e predizendo o modelo
        Verificando a acuracia do modelo
        resultado = São as inferencias que o modelo predice tendo por base o as caracteristicas e resultados passados 
        no treino marcacoes_teste = São os resultados esperados para os dados enviados para predição
    """
    modelo.fit(treino_dados, treino_marcacoes)

    resultado = modelo.predict(teste_dados)

    diferencas = (resultado == teste_marcacoes)

    acertos = sum(diferencas)
    taxa_acertos = 100 * (acertos / len(teste_dados))
                          
    print("Taxa de acertos para algoritmo {}, é de: {:.2f}" .format(nome_algoritmo, taxa_acertos))
    return taxa_acertos

In [18]:
def fit_and_predict_dados_reais(modelo, treino_dados, treino_marcacoes, validacao_dados, validacao_marcacoes):
    """
        Treinando e predizendo o modelo
        Verificando a acuracia do modelo
        resultado = São as inferencias que o modelo predice tendo por base o as caracteristicas e resultados passados 
        no treino marcacoes_teste = São os resultados esperados para os dados enviados para predição
    """
    modelo.fit(treino_dados, treino_marcacoes)
    
    resultado = modelo.predict(validacao_dados)

    diferencas = (resultado == validacao_marcacoes)

    acertos = sum(diferencas)
    print("Taxa de acertos para algoritmo real, é de: {:.2f}" .format(100 * (acertos / len(validacao_dados))))

In [30]:
"""
Criando quatro modelos: MultinomialNB, AdaBoostClassifier, OneVersusRest, OneVersusOne

Obs.: De maneira geral a previsão dos algoritmos funcionam identificando e classificando os elementos por meio
das características que foram passadas no treinamento.
Obs.1: Se a performance não for problema o algoritmo OneVersusOne deve ser executado.
"""
resultados = {}
print("Taxa de acertos base: {:.2f}" .format(taxa_acerto_base))

modelo_multinomial = MultinomialNB()
resultado_multinomial = fit_and_predict(modelo_multinomial, "MultinomialNB", treino_dados, treino_marcacoes,
                                        teste_dados, teste_marcacoes)
resultados[resultado_multinomial] = modelo_multinomial

modelo_adaBoostClassifier = AdaBoostClassifier()
resultado_adaboost = fit_and_predict(modelo_adaBoostClassifier, "AdaBoostClassifier", treino_dados, treino_marcacoes,
                                     teste_dados, teste_marcacoes)
resultados[resultado_adaboost] = modelo_adaBoostClassifier

modelo_OneVersusRest = OneVsRestClassifier(LinearSVC(random_state=0))
resultado_OneVersusRest = fit_and_predict(modelo_OneVersusRest, "OneVersusRest", treino_dados, treino_marcacoes,
                                          teste_dados, teste_marcacoes)
resultados[resultado_OneVersusRest] = modelo_OneVersusRest

modelo_OneVersusOne = OneVsOneClassifier(LinearSVC(random_state=0))
resultado_OneVersusOne = fit_and_predict(modelo_OneVersusOne, "OneVersusOne", treino_dados, treino_marcacoes,
                                          teste_dados, teste_marcacoes)
resultados[resultado_OneVersusOne] = modelo_OneVersusOne

vencedor = resultados[max(resultados)]
fit_and_predict_dados_reais(vencedor, treino_dados, treino_marcacoes, validacao_dados, validacao_marcacoes)

Taxa de acertos base: 80.00
Taxa de acertos para algoritmo MultinomialNB, é de: 88.89
Taxa de acertos para algoritmo AdaBoostClassifier, é de: 80.00
Taxa de acertos para algoritmo OneVersusRest, é de: 93.33
Taxa de acertos para algoritmo OneVersusOne, é de: 100.00
Taxa de acertos para algoritmo real, é de: 100.00


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
